In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls

drive  sample_data


In [1]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/Intelligent_parking using Mask RCNN")

In [4]:
!ls

1.15.0	images			MANIFEST.in	    output_frames     setup.cfg
assets	Intelligent_Parking.py	mask_rcnn_coco.h5   README.md	      setup.py
build	Int_Park.ipynb		mask_rcnn.egg-info  requirements.txt  Videos
dist	LICENSE			mrcnn		    samples


In [8]:
!pip3 install --upgrade tensorflow==1.3
!pip3 install --upgrade keras==2.0.8

     |████████████████████████████████| 43.6MB 64kB/s 
     |████████████████████████████████| 1.6MB 41.0MB/s 
     |████████████████████████████████| 890kB 42.0MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=d56603485253a0fbededd8d64f3356a8d8a8444014e5105f476da9c0724458a0
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.5
    Uninstalling bleach-3.1.5:
      Successfully uninstalled bleach-3.1.5
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0
     |████████████████████████████████| 276kB 2.8MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.8 which is incompatible.
  F

In [2]:
import os
import numpy as np
import cv2 as openCV
import matplotlib as plt
import mrcnn.config
import mrcnn.utils

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:461: FutureWarning: Passing (type, 1) or 

In [3]:
from mrcnn.model import MaskRCNN
from pathlib import Path

Using TensorFlow backend.


In [5]:
!nvidia-smi

Tue Jul  7 16:23:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# set up config of Mask-RCNN 
class Config_Mask_RCNN(mrcnn.config.Config):
    NAME = "coco_pretrained_model_config"
    DETECTION_MIN_CONFIDENCE = 0.6
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 80

In [5]:
# 1st level directory    
DIR_LIB = Path(".")
# Directoy of the RCNN model 
DIR_MODEL = os.path.join(DIR_LIB, "logs")

In [6]:
# Mask RCNN model path
coco_model = os.path.join(DIR_LIB, "mask_rcnn_coco.h5")

In [7]:
rcnn_model = MaskRCNN(mode = "inference", model_dir = DIR_MODEL, config = Config_Mask_RCNN())
rcnn_model.load_weights(coco_model, by_name=True)

In [8]:
# input video for test
VIDEO_INPUT = "Videos/Video.mp4"
VIDEO_OUTPUT = './Final_Video.avi'
video_frames = openCV.VideoCapture(VIDEO_INPUT)

In [9]:
# getting the bounding boxes from the frame
def get_boxes_on_vehicles(bound_boxes, class_ids):
    vehicle_boxes = []

    # iterate through all the boxes to the vehicles
    for i, box in enumerate(bound_boxes):
        if class_ids[i] in [3, 4, 8, 6]:
            vehicle_boxes.append(box)

    return np.array(vehicle_boxes)

In [10]:
# parking slots spotted
parked_vehicle_boxes = None
# Loop over all frames
frame_count = 0
 # Assume no spaces are free until we find one that is free
free_space = False
video_writer = None

In [12]:
while video_frames.isOpened():
    success, frame = video_frames.read()

    if not success:
        print("Last frame processed or video format is corrupted")
        break

    elif frame_count < 60:
      # compare two frames to cancel the detection of moving vehicle
      success, next_frame = video_frames.read()
      diff_frames = openCV.absdiff(frame, next_frame)  
      grey_motion = openCV.cvtColor(diff_frames, openCV.COLOR_BGR2GRAY)
      blur_pixel = openCV.GaussianBlur(grey_motion, (1, 1), 0)
      ret, threshold = openCV.threshold(blur_pixel, 20, 255, openCV.THRESH_BINARY)
      
      # erode the car which is moving so that it is not detected by MASKRCNN.
      dilated = openCV.dilate(threshold, np.ones((10, 10), np.uint8), iterations = 1 )
      eroded = openCV.erode(dilated, np.ones((10, 10), np.uint8), iterations = 1 )

      # processed frame by adding contours so that MaskRCNN doesn't get applied
      c, h = openCV.findContours(eroded, openCV.RETR_TREE, openCV.CHAIN_APPROX_SIMPLE)
      next_frame = openCV.drawContours(next_frame, c, -1, (0,0,0), openCV.FILLED)

      if frame_count%5 == 0:
        print("Current frame number : " + str(frame_count))
        plt.pyplot.imshow(next_frame)
        plt.pyplot.show()
        
      frame_count = frame_count + 1
      continue 

    # Converting the BGR color to RGB color.     
    rgb_image = frame[:, :, ::-1] 

    # Detect cars using Mask RCNN
    if parked_vehicle_boxes is None:
        print("Marking vehicles. Frame number:  ", frame_count)
        results = rcnn_model.detect([rgb_image], verbose=0)
        res = results[0]
        parked_vehicle_boxes = get_boxes_on_vehicles(res['rois'], res['class_ids'])

    elif frame_count%200 == 0 and len(parked_vehicle_boxes) != 0:
        results = rcnn_model.detect([rgb_image], verbose=0)
        res = results[0]

        # Get where vehicles are currently located in the frame
        vehicle_boxes = get_boxes_on_vehicles(res['rois'], res['class_ids'])

        # how much those vehicles overlap with the known parking slots
        overlaps = mrcnn.utils.compute_overlaps(parked_vehicle_boxes, vehicle_boxes)

        # For all the parking slots found around parked vehicles
        for parking_slot, overlap_areas in zip(parked_vehicle_boxes, overlaps):

            # find the max amount it was covered by any
            # vehicle detected for this slot
            max_IoU_overlap = np.max(overlap_areas)

            # corner co-ordinates of the parking slot
            y1, x1, y2, x2 = parking_slot

            # vehicle has left the slot
            if max_IoU_overlap < 0.15:
                # Parking slot is empty
                openCV.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                formatedText = "Parking Slot free : "
                openCV.putText(frame, formatedText, (x1 + 6, y2 - 6), openCV.FONT_HERSHEY_SIMPLEX, 0.2, (255, 255, 255))
                free_space = True
            else:
                # Parking slot is still occupied 
                openCV.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)

            # Write the IoU measurement inside the box
            # font = openCV.FONT_HERSHEY_DUPLEX
            # openCV.putText(frame, f"{max_IoU_overlap:0.2}", (x1 + 6, y2 - 6), font, 0.7, (255, 255, 255))

    img = str(frame_count) + ".jpg"
    img = os.path.join('./output_frames', img)
    openCV.imwrite(img, frame)
    frame_count+=1

    #if video_writer is None:
		# Saving the Video
	  #  fourcc = openCV.VideoWriter_fourcc(*"MJPG")
	  #  video_writer = openCV.VideoWriter(VIDEO_OUTPUT, fourcc, 15, (frame.shape[1], frame.shape[0]), True)
    #video_writer.write(frame)

print("Video completed")
# video_writer.release()
video_frames.release()

Last frame processed or video format is corrupted
Video completed


In [14]:
#create video including all the frames in the ak folder
import glob

images = list(glob.iglob(os.path.join('./output_frames', '*.*')))
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))

# Get all image file paths to a list.
# Sort the images by name index.
# images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))

def make_video(outvid, images=None, fps=30, size=None,
               is_color=True, format="FMP4"):
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
    """
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid
  
make_video('./FinalVideo.mp4', images, fps=30)

<VideoWriter 0x7f1a47f2db90>